In [2]:
import pandas as pd
import numpy as np

coin_name = "KRW-XRP"
# df_org = pd.read_csv('./data/minute_index_result.csv') #예측모델
df_org = pd.read_csv("./data/{}-index.csv".format(coin_name))
df_org.columns 

Index(['t', 'o', 'h', 'l', 'c', 'v', 'sma7', 'sma14', 'sma99', 'wma7', 'wma25',
       'wma99', 'ema7', 'ema25', 'ema99', 'macd', 'macd_s', 'macd_d', 'rsi7',
       'rsi14', 'rsi28', 'srsi', 'srsik', 'srsid', 'bh', 'bhi', 'bl', 'bli',
       'bm', 'bw', 'vwap'],
      dtype='object')

In [6]:
#단계별 구매 수량
def get_buy_amt_list(buy_amt_unit, buy_cnt_limit, increace_rate):
    buy_amt = 0
    buy_amt_list = [0.0]
    for idx in range(0, buy_cnt_limit):
        temp_amt = buy_amt_unit + buy_amt * increace_rate
        buy_amt = round(buy_amt + temp_amt, 4)
        buy_amt_list.append(buy_amt)
    return buy_amt_list

#손실 최소화 실현 금액 계산
def get_max_loss(close, buy_amt_unit, buy_cnt_limit, increace_rate, max_loss_rate):
    buy_amt = 0
    buy_price = 0
    for idx in range(0, buy_cnt_limit):
        temp_amt = buy_amt_unit + buy_amt * increace_rate
        buy_price = round(buy_price + close * temp_amt, 4)
        buy_amt = round(buy_amt + temp_amt, 4)
    return round(buy_price * max_loss_rate, 4)

revenue_rate = 0.014  #익절 비율
max_loss_rate = 0.2   #손절 비율: 20%
increace_rate = -0.4  
buy_cnt_limit = 7   #최대 오픈 건수
buy_amt_unit = 58     #최소 오픈 수량
trade_fee = 0.001 #거래수수료
close = 1300
buy_amt_list = get_buy_amt_list(buy_amt_unit, buy_cnt_limit, increace_rate)
max_loss = get_max_loss(close, buy_amt_unit, buy_cnt_limit, increace_rate, max_loss_rate)
print("*buy_amt_list:{} max_loss:{}".format(buy_amt_list, max_loss))

buy_cnt = 0 
buy_price = 0
buy_amt = 0
revenue = 0
revenue_t = 0

df = df_org.iloc[df_org.shape[0]-14400:,]
# df = df_org2
for i in range(0, df.shape[0]-1):
    close1 = round(df.iloc[i:i+1,]['c'].values[0],4)
    close2 = round(df.iloc[i+1:i+2,]['c'].values[0],4) #다음 분의 종가로 산다
    wma25 = round(df.iloc[i:i+1,]['wma25'].values[0],4)
    wma99 = round(df.iloc[i:i+1,]['wma99'].values[0],4)
    vwap = round(df.iloc[i:i+1,]['vwap'].values[0],4)
    macd_s = round(df.iloc[i:i+1,]['macd_s'].values[0],4)
    macd_d = round(df.iloc[i:i+1,]['macd_d'].values[0],4)
    
    #stop loss
    loss = buy_price - close2*buy_amt
    if loss > max_loss:
        revenue_t = close2*buy_amt - buy_price - buy_price * trade_fee
        revenue = round(revenue + revenue_t,4)
        print("*[{}] stop loss:{} revenue:{} ".format(i, revenue_t, revenue))        
        buy_cnt = 0
        buy_amt = 0
        buy_price = 0
        continue

    #take profit
    tp_revenue = close2*buy_amt - (buy_price + buy_price*revenue_rate)
    if buy_cnt > 0 and tp_revenue > 0:
        revenue_t = close2*buy_amt - buy_price - buy_price * trade_fee
        revenue = round(revenue + revenue_t,4)
#         print("*[{}] take profit:{} revenue:{} ".format(i, revenue_t, revenue))        
        buy_cnt = 0
        buy_amt = 0
        buy_price = 0
        continue
    
    #buy
    if buy_cnt < buy_cnt_limit and close1 > vwap and macd_d > macd_s and wma25 > wma99: #로직컬
        temp_amt = buy_amt_unit + buy_amt*increace_rate
        buy_price = round(buy_price + close2*temp_amt,4)
        buy_amt = round(buy_amt + temp_amt,4)
        buy_cnt = buy_cnt + 1 
    
#     print("c1:{} c2:{} vwap:{} olc:{} ola:{} olp:{} loss:{} tp_revenue:{} revenue:{}"
#       .format(close1, close2, vwap, buy_cnt, buy_amt, buy_price, loss, tp_revenue, revenue))
        
        
print(revenue)


*buy_amt_list:[0.0, 58.0, 92.8, 113.68, 126.208, 133.7248, 138.2349, 140.9409] max_loss:36644.6392
*[14306] stop loss:-37074.00739980001 revenue:22503.2895 
22503.2895


In [16]:
272015.8648

272015.8648